In [1]:
import spacy as spacy
#check https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy
import re
import os.path
my_path = os.path.abspath('')

In [2]:

import requests
# pprint is used to format the JSON response
from pprint import pprint
import os

subscription_key = "cda56a1fa0754d4b95974292b33a18aa"
endpoint = "https://entities-extract-test.cognitiveservices.azure.com/"
entities_url = endpoint + "/text/analytics/v3.0/entities/recognition/general/recognition/general"

In [3]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(subscription_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [4]:
def cloud_get_num_entity_recognition(client, string):

    try:
        ent_counter=0
        documents = [string]
        result = client.recognize_entities(documents = documents)[0]

        #print("Named Entities:\n")
        for entity in result.entities:
            if entity.category=="Organization" or entity.category=="Product":
                #print("\tText: \t", entity.text, "\tCategory: \t", entity.category, "\tSubCategory: \t", entity.subcategory,"\n\tConfidence Score: \t", round(entity.confidence_score, 2), "\n")
                ent_counter=ent_counter+1
        return ent_counter
    except Exception as err:
        print("Encountered exception. {}".format(err))
        
        
        


In [5]:
#read all the text file in the directory
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))       
    return files

#print elements from list
def print_list(lists):
    for list in lists:
        print(list)

In [9]:
def get_num_of_company_competitor_ent(doc):
    relations = list()
    num_ent_competitor=0

    for sent in doc.sents:
        check=0
        for token in sent:
            if token.lemma_ =="competitor" or token.lemma_ =="compete" or token.lemma_ =="competition":
                if check==0: 
                    num_ent_competitor= num_ent_competitor+ cloud_get_num_entity_recognition(client, sent.text)

                check=check+1
            
    return num_ent_competitor
#products
def get_num_of_company_product_ent(doc):
    
    relations = list()
    num_ent_product=0
    has_other_relation=False

    for sent in doc.sents:
        check=0
        for token in sent:
            if token.lemma_ =="competitor" or token.lemma_ =="compete" or token.lemma_ =="competition" or token.lemma_=="client" or token.lemma_=="manufacturer" or token.lemma_=="customer" or token.lemma_=="buyer" or token.lemma_=="purchase" or token.lemma_=="acquisition" or token.lemma_=="acquire" or token.lemma_=="merger" or token.lemma_=="merge" or token.lemma_=="consolidate" or token.lemma_=="consolidation":
                has_other_relation=True
            if has_other_relation==False:
                if token.lemma_ =="product" or token.lemma_ =="service" or token.lemma_ =="sell" or token.lemma_=="device" or token.lemma_=="provide" or token.lemma_=="offer" or token.lemma_=="develop" or token.lemma_=="design" or token.lemma_=="solutions"or token.lemma_ =="manufacture" or token.lemma_ =="tool":
                    if check==0: 
                        num_ent_product= num_ent_product+ cloud_get_num_entity_recognition(client, sent.text)
                    check=check+1
            
    return num_ent_product

#products
def get_num_of_company_aquisition_ent(doc):
    
    relations = list()
    num_ent_aquisition=0

    for sent in doc.sents:
        check=0
        for token in sent:
            if token.lemma_=="acquisition" or token.lemma_=="acquire" or token.lemma_=="merger" or token.lemma_=="merge" or token.lemma_=="consolidate" or token.lemma_=="consolidation":
                if check==0: 
                    num_ent_aquisition=num_ent_aquisition+cloud_get_num_entity_recognition(client, sent.text)
                check=check+1
            
    return num_ent_aquisition



def get_competitor_triples(triples, doc):
 
    for sent in doc.sents:
        #get relations
        relations = list()
        ents=list()
        head=list()
        tail=list()
        check=True
        has_other_relation=False
        for i, token in enumerate(sent):
            if token.lemma_ =="competitor" or token.lemma_ =="compete" or token.lemma_ =="competition" and len(sent.ents)>1:
                relations.append(token.text)
                if check==True:
                    for ent in sent[:i].ents:
                        if ent.label_=='ORG' or ent.label_=='PRODUCT':
                            if ent.text not in head and ent.text not in tail:
                                head.append([ent.text,ent.label_])
                    for ent in sent[i:].ents:
                        if ent.label_=='ORG' or ent.label_=='PRODUCT':
                            if ent.text not in tail and ent.text not in head:
                                tail.append([ent.text,ent.label_])
                    check= False
                    if len(head)==0 and len(tail)>1:
                        head.append(tail[0])
                        tail.pop(0)
                    #if len(head)>1 and len(tail)==0:
                    
                    
        #if len(relations)>0 and len(head)>0 and len(tail)>0:

            #print(head )
            #print(relations)
            #print(tail)
            #print()
        #form triples  
        if len(relations)>0 and len(head)>0 and len(tail)>0:
            for h in head:
                for t in tail:
                    triples.append([h, relations[0], t])
    #print(triples)
    return triples


In [10]:
nlp = spacy.load('en_core_web_lg')
nlp.remove_pipe("ner")

files = file_reader(my_path+"/clean_data/item1_business")
files.sort()
print_list(files)

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/ACIW_section_2.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/ADS_section_3.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/AKAM_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/ALRM_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/AMZN_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/BCOR_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/BKI_section_1.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/clean_data/item1_business/CARG_section_1.txt
/Users/dav

In [11]:
competitor_triples=list()
product_triples=list()
aquisition_triples=list()
num_aquisition_ent=0
num_of_company_product_ent=0
num_of_company_competitor_ent=0
for file in files:
    text=open(file)
    doc = nlp(text.read())
    num_of_company_competitor_ent=num_of_company_competitor_ent+get_num_of_company_competitor_ent(doc)
    num_of_company_product_ent=num_of_company_product_ent+get_num_of_company_product_ent(doc)
    num_aquisition_ent=num_aquisition_ent+get_num_of_company_aquisition_ent(doc)
    #aquisition_triples=get_aquisition_tuples(aquisition_triples,doc)
    #product_triples=get_product_tuples(product_triples,doc)
    #competitor_triples=get_competitor_triples(competitor_triples, doc)

In [12]:

print(num_aquisition_ent)
print(num_of_company_product_ent)
print(num_of_company_competitor_ent)

1840
367
1763


In [ ]:
"""
cloud_get_num_entity_recognition(client,"Twitter Inc is what’s happening in the world and what people are talking about right now.")

"""